# Introduction

In this notebook, I will analyse the Seattle AirBNB Dataset from [Inside AirBnb](http://insideairbnb.com/). I will follow the CRISP-DM process when analysing their data. The CRISP-DM stages are:
- Business Understanding
- Data Understanding
- Data Preparation
- Modelling
- Evaluation
- Deployment

## Imports

In [1]:
import pandas as pd
import numpy as np
import pandas_profiling as pp
import altair as alt
import matplotlib.pyplot as plt
from geopy.distance import geodesic
from multiprocessing import Pool
from multiprocessing_fn import distance_method
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import FunctionTransformer, LabelEncoder
import lightgbm as lgb
import _pickle as cPickle
import copy 

pd.set_option("display.max_columns",50)
alt.data_transformers.disable_max_rows()
%matplotlib inline

# Business and Data Understanding

I will answer the following 4 business questions:
1. Does becoming a verified host positively impact your monthly AirBnb income or occupancy rate in Seattle?
1. Does becoming a Super-Host positively impact your monthly AirBnb income or occupancy rate in Seattle?
1. Does proximity to competition negatively impact your monthly AirBnb income?
1. What are the key factors that influence the rate a host sets for their listing in Seattle?

In [2]:
data_dir = '../data'
raw_dir = data_dir + '/raw'
processed_dir = data_dir + '/processed'

reviews_df = pd.read_csv(raw_dir + '/reviews.csv')
calendar_df = pd.read_csv(raw_dir + '/calendar.csv')
listings_df = pd.read_csv(raw_dir + '/listings.csv')

reviews_df['date'] = pd.to_datetime(reviews_df['date'])
calendar_df['date'] = pd.to_datetime(calendar_df['date'])
listings_df['last_scraped'] = pd.to_datetime(listings_df['last_scraped'])
listings_df['host_since'] = pd.to_datetime(listings_df['host_since'])

NB: The commented task below is a long running task. It can take up to **15 min** to run.

In [3]:
# pp.ProfileReport(reviews_df).to_file(raw_dir + '/reviews_profile_report.html')

# pp.ProfileReport(calendar_df).to_file(raw_dir + '/calendar_profile_report.html')

# pp.ProfileReport(listings_df).to_file(raw_dir + '/listings_profile_report.html')

I used the Profile Reports from Pandas Profiling as well as this [data dictionary](../../data/raw/Inside%20Airbnb%20Data%20Dictionary.xlsx) from InsideAirBnB to understand the Data. The profile reports can be found here:
- [reviews_profile_report](../../data/raw/reviews_profile_report.html)
- [calendar_profile_report](../../data/raw/calendar_profile_report.html)
- [listings_profile_report](../../data/raw/listings_profile_report.html)

# Data Preparation

## Cleaning Dataset
 
 In this section, I fill nulls, drop unnecessary columns and correct datatypes

In [4]:
# Selecting the columes required for the analysis
listings_filtered_df = listings_df[['name', 'description', 'host_id','host_response_time', 'host_since', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'latitude',
       'longitude', 'neighbourhood_group_cleansed', 'property_type', 'room_type', 'accommodates', 'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'availability_30', 'availability_60', 'availability_90', 'availability_365', 'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',  'review_scores_communication', 'review_scores_location', 'review_scores_value', 'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms', 'reviews_per_month']].copy()

listings_filtered_df[['host_has_profile_pic', 'host_is_superhost', 'host_identity_verified']] = listings_filtered_df[['host_has_profile_pic', 'host_is_superhost', 'host_identity_verified']].replace({'t':True, 'f':False})

# Filling Nulls
for col in ['host_response_time', 'host_response_rate', 'host_acceptance_rate', 'bedrooms', 'beds', 'bathrooms_text']:
    listings_filtered_df[col] = listings_filtered_df[col].fillna(listings_filtered_df[col].mode().iloc[0])
    
zero_cols = ['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']    
listings_filtered_df[zero_cols] = listings_filtered_df[zero_cols].fillna(0)

listings_filtered_df['description'] = listings_filtered_df['description'].fillna('')
listings_filtered_df['host_since'] = pd.to_datetime(listings_filtered_df['host_since'])

In [5]:
listings_filtered_df = listings_filtered_df.assign(host_response_rate=listings_filtered_df['host_response_rate'].replace('[\%]', '', regex=True).astype(int))
listings_filtered_df = listings_filtered_df.assign(host_acceptance_rate=listings_filtered_df['host_acceptance_rate'].replace('[\%]', '', regex=True).astype(int))
listings_filtered_df = listings_filtered_df.assign(bathrooms_text=listings_filtered_df['bathrooms_text'].replace('Half|half', '0.5', regex=True).replace('[\sA-Za-z\-]+', '', regex=True).astype(float))

listings_filtered_df = listings_filtered_df.assign(price=listings_filtered_df['price'].replace('[\$\,]', '', regex=True).astype(float))

## Feature Extraction

In [6]:
def item_list_counter(items_series):
    """
    This method takes a series that contains elements with a list of strings and returns a dictionary that contains the total number of times each string in the lists 
    is mentioned in the entire series.
    
    :param series: Pandas Series object
    :return: dict 
    """
    items_list = items_series.apply(eval).tolist()
    counts_dict = {}
    for i, items in enumerate(items_list):
        for item in items:
            if counts_dict.get(item, None) is None:
                counts_dict[item] = 1
            else:
                counts_dict[item] += 1
    return counts_dict

### Derive Occupancy, Income per month and Nights per year 

In this section, I use the occupancy model described [here](http://insideairbnb.com/about.html#disclaimers)  or [here](http://insideairbnb.com/seattle/?neighbourhood=&filterEntireHomes=false&filterHighlyAvailable=false&filterRecentReviews=false&filterMultiListings=false#) to calculate the mentioned features. 

In [7]:
listings_filtered_df['estimated_bookings'] = listings_filtered_df.number_of_reviews_ltm * 2

listings_filtered_df['avg_length_of_stay'] = [min_nights if min_nights > 3 else 3 for min_nights in  listings_filtered_df['minimum_nights']]
uncapped_nights_per_year  = listings_filtered_df['estimated_bookings'] * listings_filtered_df['avg_length_of_stay']

# By limiting the nights to 255 in a year, we limit the occupancy to 70% as recommended by InsideAirBnB
listings_filtered_df['nights_per_year'] = [255 if nights > 255 else nights for nights in uncapped_nights_per_year]
listings_filtered_df['occupancy_rate'] = listings_filtered_df['nights_per_year']/365*100
listings_filtered_df['income_per_month']  = listings_filtered_df['nights_per_year']*listings_filtered_df['price']/12

### Grouping Amenities

In this section, I categorise the Amenities that are available in each listing into 30 bin. I follow the following 3 steps:
1. Rank the amenities based on the number of mentioned and save the data to **ranking_of_ammenites.csv**
1. Manually create categories for these amenites using Regex and save them to **ammenity_categories.txt**
1. Group the listings in the listings dataset and create a new set of features to append to `listings_filtered_df`

In [8]:
# Here we rank all amenities based on how often they are mentioned. We will use this to build  the groups
amenity_counter = item_list_counter(listings_filtered_df.amenities)
ammenity_counts_df = pd.DataFrame({'ammenity':amenity_counter.keys(), 'counts':amenity_counter.values()})
ammenity_counts_df.sort_values(by=['counts'], ascending=False).to_csv(processed_dir + "/ranking_of_ammenites.csv")

In [9]:
%%time
# Here we group the amenities and create a set of new features for the listings dataset
with open(processed_dir + '/ammenity_categories.txt') as fp:
    out = fp.read()

categories = []
categories
for string_ in out.split('\n'):
    if string_ == '':
        break
    categories.append(tuple(string_.split(':')))

amenities = listings_filtered_df.amenities.apply(eval).tolist()

row_list = []
row = {}
for amen in amenities:
    for item, categ_name in categories:
        row[categ_name] = pd.Series(amen).str.contains('(?i)'+item, regex=True).sum()
        
    row_list.append(copy.deepcopy(row))
    
amenities_df = pd.DataFrame(row_list)
amenities_df['TV'] = amenities_df['TV'] - amenities_df['TV_Entertainment']  # We separate TV with entertainment options from a TV set amenity
amenities_df.loc[amenities_df['TV'] < 0, 'TV'] = 0

amenities_df.columns=['amen_'+col for col in amenities_df.columns]
amenities_df.sample(5)

Wall time: 43.1 s


,amen_TV,amen_TV_Entertainment,amen_Sound_System,amen_WorkSpace,amen_Internet,amen_Paid_Parking,amen_Free_Parking,amen_Clothing_Storage,amen_Linens,amen_House_Keeping,amen_Bathroom_Essentials,amen_Restuarant,amen_Breakfast,amen_Gym,amen_Pool_Sauna,amen_Games,amen_Children_Babies,amen_Dryer_Washer,amen_Backyard_Garden,amen_Beach_Water_Front,amen_Pets,amen_Kitchen,amen_Kitchen_Appliances,amen_Cooking_Dishes,amen_Host_Greeting,amen_Long_Term_Stay,amen_Indoor_Fireplace,amen_Outdoor,amen_Air_Conditioning,amen_Heating,amen_Safety_Security_First_Aid,amen_Staff
2164,0,2,0,0,1,0,0,1,2,1,2,0,0,1,0,0,0,3,0,0,0,1,5,1,0,1,0,1,0,1,3,0
1680,1,0,0,0,1,0,1,1,2,1,2,0,0,0,0,0,0,3,0,0,0,1,0,1,0,1,0,0,0,1,5,0
1951,1,0,0,1,1,1,1,1,1,1,3,0,0,0,0,0,2,3,0,0,0,1,5,2,0,1,0,0,0,1,5,0
1156,0,2,0,1,1,1,0,1,2,1,2,0,0,0,0,1,0,3,0,0,0,0,4,2,0,1,0,0,1,0,3,0
3802,1,0,0,1,2,0,2,2,2,0,5,1,0,0,0,0,0,4,0,0,0,1,9,5,0,2,0,4,1,1,6,0


### Proximity and Similarity Features

In this section, I categorise the listings based on their proximity and similarity to other listings. I take the following steps:
1. Create list of boolean lists that indicate all the similar listings for a given listings. Here, I define similar listings as those that have the same**property_type, bedrooms and beds**.
1. Next I get the distance between all listings using `distance_method()` and the multiprocessing module. These distances are then combined with the boolean lists to get the listings that are close to a listing and also similar to it.

In [10]:
# Picking similar listings based on property ty
similar_locations_bool = [(listings_filtered_df[['property_type', 'bedrooms', 'beds']] == listings_filtered_df[['property_type', 'bedrooms', 'beds']].iloc[i]).sum(axis=1) >= 3 for i in range(listings_filtered_df.shape[0])]

NB: The commented task below is a long running task. It can take up to **16 min** to run.

In [11]:
%%time
# index = listings_filtered_df.index.tolist()
# lat_long_values = listings_filtered_df[['latitude','longitude']].values
# list_of_lists = list(zip(list(lat_long_values), [lat_long_values]*len(lat_long_values), similar_locations_bool))

# pool = Pool(8)
# distances_list = pool.map(distance_method, list_of_lists)
# pool.close()

Wall time: 0 ns


In [12]:
# distances_df = pd.DataFrame(distances_list, columns=['dist_500m', 'dist_1500m', 'dist_4000m'])

Loading cached distances dataframe from local storage.

In [13]:
# distances_df.to_csv(processed_dir + "/distances_df_new.csv")
distances_df = pd.read_csv(processed_dir + "/distances_df_new.csv")

In [14]:
distances_df.sample(5)

,dist_500m,dist_1500m,dist_4000m
2187,1,1,1
1314,1,8,72
1407,5,20,58
3045,18,137,310
305,1,1,5


### Merging all Features

In this section, I drop unnecessary columns and merge the `amenites_df` and `distances_df` with the `listings_filtered_df`.

In [15]:
listings_filtered_df = listings_filtered_df.drop(columns=['amenities', 'host_verifications', 'latitude', 'longitude', 'description', 'minimum_nights', 'number_of_reviews_ltm', 'host_id', 'host_listings_count', 'estimated_bookings', 'nights_per_year'])

In [16]:
listings_merged_df = pd.concat([listings_filtered_df, amenities_df, distances_df], axis=1)

In [17]:
listings_merged_df.sample(5)

,name,host_response_time,host_since,host_response_rate,host_acceptance_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,neighbourhood_group_cleansed,property_type,room_type,accommodates,bathrooms_text,bedrooms,beds,price,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,...,amen_Bathroom_Essentials,amen_Restuarant,amen_Breakfast,amen_Gym,amen_Pool_Sauna,amen_Games,amen_Children_Babies,amen_Dryer_Washer,amen_Backyard_Garden,amen_Beach_Water_Front,amen_Pets,amen_Kitchen,amen_Kitchen_Appliances,amen_Cooking_Dishes,amen_Host_Greeting,amen_Long_Term_Stay,amen_Indoor_Fireplace,amen_Outdoor,amen_Air_Conditioning,amen_Heating,amen_Safety_Security_First_Aid,amen_Staff,dist_500m,dist_1500m,dist_4000m
2054,Day 1 | Ultra Modern DT Studio | 1,within an hour,2017-04-06,100,100,False,True,True,Downtown,Entire serviced apartment,Entire home/apt,2,1.0,1.0,1.0,215.0,30,60,90,180,3,0,3.00,5.00,5.00,...,1,0,0,1,0,0,0,3,0,0,0,1,0,0,0,1,0,0,0,1,3,0,25,50,75
13,Suite+Office in Gorgeous View Home,within a few hours,2011-05-05,100,47,True,True,True,West Seattle,Private room in residential home,Private room,2,1.0,1.0,1.0,100.0,0,0,0,260,143,0,4.92,4.94,4.95,...,2,0,0,0,0,0,0,4,0,0,0,1,5,2,0,1,1,1,1,1,4,0,1,2,21
1075,Modern Apartment in Queen Anne with Parking,within an hour,2015-11-24,100,100,True,True,True,Interbay,Entire rental unit,Entire home/apt,3,1.0,1.0,2.0,89.0,0,0,30,179,194,0,4.91,4.94,4.95,...,2,0,0,0,0,0,1,4,0,0,0,1,5,2,0,1,0,0,0,1,5,0,2,10,59
1217,"Queen Anne Bungalow - 2Bed/2Bath,AC, Full Kitchen",within an hour,2014-08-17,100,100,True,True,True,Queen Anne,Entire rental unit,Entire home/apt,4,2.0,2.0,2.0,185.0,0,14,38,165,329,5,4.99,4.97,5.00,...,6,0,0,0,1,0,0,4,0,0,0,1,9,4,0,2,0,2,1,1,5,0,2,18,92
2429,Garden Apartment at The Old Orchard House,within a day,2016-01-07,100,71,False,True,False,University District,Entire rental unit,Entire home/apt,6,1.0,3.0,3.0,400.0,5,35,65,340,11,2,5.00,5.00,5.00,...,4,1,0,0,0,2,1,3,1,1,0,1,9,5,0,2,0,4,1,1,7,0,1,2,6


In [18]:
listings_merged_df.to_csv(processed_dir + '/listings_merged_df.csv', index=False)

NB: The commented task below is a long running task. It can take up to **8 min** to run.

In [19]:
%%time
# Evaluating cleaned dataset with Pandas Profiling
# pp.ProfileReport(listings_merged_df,
#     title="Cleaned Listings Report",
#     correlations={
#         "pearson": {"calculate": True},
#         "spearman": {"calculate": True},
#         "kendall": {"calculate": False},
#         "phi_k": {"calculate": True},
#         "cramers": {"calculate": False},
#     },plot={
#         "dpi": 1000, "image_format": "png"
#     }, interactions={
#         "targets":['income_per_month', 'dist_500m', 'occupancy_rate', 'review_scores_communication']
#     }, missing_diagrams=None ,progress_bar=True).to_file(processed_dir + '/listings_merged_profile_report.html')

Wall time: 0 ns


Access the `listing_merged_df` here [listing_merged_profile_report](../../data/processed/listings_merged_profile_report.html)

# Analysis and Modelling

In this section, I attempt to answer the 4 business questions using the cleaned data

In [20]:
def perform_permutation_test(group_1, group_2, permutations=10_000, confidence_interval=95, x_axis_title='', chart_title=''):
    """
    This method performs a permutation test to generate a sampling distribution of the mean differences.
    
    We then calculate the p-value of the test_statistic and use this and the confidence_interval to check for statistical significance.
    
    :param group_1: 
    :param group_2:
    :param permutations: Number of permutations to be performed
    :param confidence_interval: Confidence interal to use when performing the test
    :return: The calculated P-Value, The mean_differences that were generated from the permutation test
    """
    print("Our null hypothesis: The difference in mean values is due to random chance")
    group_1 = list(group_1)
    group_2 = list(group_2)
    mean_group_1= np.mean(group_1)
    mean_group_2= np.mean(group_2)
    test_statistic = mean_group_1 - mean_group_2
    print(f"Our test statistic is {test_statistic:,.4f}")
    
    full_set = group_1 + group_2
    proportions_group_1 = len(group_1)/len(full_set)
    print(f"Group 1 has a proportion of {proportions_group_1:.4f}")
    threshold = (1-proportions_group_1)
    
    # We perform the permutations
    mean_differences = []
    for i in range(permutations):
        # empty array to represent a random total sales w/ profile pic
        group_1_random = []
        # empty array to represent a random total sales w/o profile pic
        group_2_random = []

        # use a for loop to randomonly place the total sales in one of the two lists
        for item in full_set:
            random_value = np.random.random()
            # randomally assinging each value to a new list
            if random_value > threshold:
                group_1_random.append(item)
            elif random_value <= threshold:
                group_2_random.append(item)

        mean_group_1 = np.mean(group_1_random)
        mean_group_2 = np.mean(group_2_random)
        # find the itteration mean and append to our mean difference list
        iteration_mean_difference =  mean_group_1 - mean_group_2
        mean_differences.append(iteration_mean_difference)

    # We determine the P-Value
    total = 0
    for mean_diff in mean_differences:
        if mean_diff >= test_statistic:
            total += 1

    #p value calculated by calulating percentage of values greater then the mean difference
    p_value = total / permutations
    print(f"The p-value for these 2 groups is {p_value:.8f}")
    
    conf_interval = 1-confidence_interval/100
    if p_value > conf_interval :
        print(f"We fail to reject the Null Hypothesis")
    else:
        print(f"We reject the Null Hypothesis")
        
    chart = alt.Chart(pd.DataFrame(mean_differences, columns=['Diffs'])).mark_bar().encode(x=alt.X('Diffs', bin=alt.Bin(maxbins=15), title=f"Monthly differences of {x_axis_title}"), y=alt.Y('count()', title='Count')).properties(title=f"Sampling Distribution of {chart_title}", width=600)
            
    return p_value, chart

In [21]:
def clean_fit_lgbm_mod(df, response_col, cat_cols, dummy_na, test_size=.3, rand_state=42):
    '''
    INPUT:
    df - a dataframe holding all the variables of interest
    response_col - a string holding the name of the column 
    cat_cols - list of strings that are associated with names of the categorical columns
    dummy_na - Bool holding whether you want to dummy NA vals of categorical columns or not
    test_size - a float between [0,1] about what proportion of data should be in the test dataset
    rand_state - an int that is provided as the random state for splitting the data into training and test 
    
    OUTPUT:
    test_score - float - r2 score on the test data
    train_score - float - r2 score on the test data
    lm_model - model object from lightgbm
    X_train, X_test, y_train, y_test - output from sklearn train test split used for optimal model
    '''
    #Split into explanatory and response variables
    X = df.drop(response_col, axis=1)
    y = df[response_col]

    #Split into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=rand_state)

    lgbm = lgb.LGBMRegressor(random_state=rand_state, n_estimators=100,
                         n_jobs=-1, verbose=1)\
    .fit(X_train, y_train, eval_set=[(X_test, y_test)],
         early_stopping_rounds=10,
         feature_name=list(df.drop(response_col, axis=1).columns),
         categorical_feature=cat_cols)

    #Predict using your model
    y_test_preds = lgbm.predict(X_test, num_iteration=lgbm.best_iteration_)
    y_train_preds = lgbm.predict(X_train, num_iteration=lgbm.best_iteration_)

    #Score using your model
    test_score = r2_score(y_test, y_test_preds)
    train_score = r2_score(y_train, y_train_preds)

    return test_score, train_score, lgbm, X_train, X_test, y_train, y_test

## Does becoming verified as a host positively impact your monthly AirBnb income or occupancy rate in Seattle?


In [22]:
%%time
output = perform_permutation_test(listings_merged_df[listings_merged_df.host_identity_verified==True].income_per_month , listings_merged_df[listings_merged_df.host_identity_verified==False].income_per_month, x_axis_title='Monthly Income', chart_title='Monthly Income differences between Verified and Unverified Hosts')
output[1]

Our null hypothesis: The difference in mean values is due to random chance
Our test statistic is 52.8465
Group 1 has a proportion of 0.8447
The p-value for these 2 groups is 0.21410000
We fail to reject the Null Hypothesis
Wall time: 18.5 s


alt.Chart(...)

In [23]:
%%time
output = perform_permutation_test(listings_merged_df[listings_merged_df.host_identity_verified==True].occupancy_rate , listings_merged_df[listings_merged_df.host_identity_verified==False].occupancy_rate, x_axis_title='Occupancy Rate', chart_title='Occupancy Rate differences between Verified and Unverified Hosts')
output[1]

Our null hypothesis: The difference in mean values is due to random chance
Our test statistic is 0.1024
Group 1 has a proportion of 0.8447
The p-value for these 2 groups is 0.45090000
We fail to reject the Null Hypothesis
Wall time: 19 s


alt.Chart(...)

There is **no significant difference** between the returns of Verified and Un-verified hosts in Seattle

## Does becoming a Super-Host positively impact your monthly AirBnb income or occupancy rate in Seattle?


In [24]:
listings_merged_df['Hosted for'] =(pd.to_datetime('2021-10-17') - listings_merged_df['host_since']).dt.days/365

In [25]:
print("Non-matching Hosted for distribution. We need to resample both groups to give them a similar distribution")
alt.Chart(listings_merged_df[['Hosted for', 'host_is_superhost']]).mark_bar().encode(x=alt.X('Hosted for', bin=True), y='count()', column='host_is_superhost:N')

Non-matching Hosted for distribution. We need to resample both groups to give them a similar distribution


alt.Chart(...)

In [26]:
bins = [0, 2, 4, 6, 8, 10, 12, 14]
labels = [2, 4, 6, 8, 10, 12, 14]
listings_merged_df['hosted_for_grouped'] = pd.cut(listings_merged_df['Hosted for'], bins, labels=labels).astype(int)
super_hosts_df = listings_merged_df[listings_merged_df['host_is_superhost'] == True]
non_super_hosts_df = listings_merged_df[listings_merged_df['host_is_superhost'] == False]
per_distribution_of_superhosts = super_hosts_df.hosted_for_grouped.value_counts().reset_index().sort_values(by='index').reset_index(drop=True)
per_distribution_of_non_superhosts = non_super_hosts_df.hosted_for_grouped.value_counts().reset_index().sort_values(by='index').reset_index(drop=True)
per_distribution = pd.concat([per_distribution_of_superhosts.rename(columns={'hosted_for_grouped':'superhost', 'index':'Age Group'}), per_distribution_of_non_superhosts.rename(columns={'hosted_for_grouped':'non_superhost'}).drop(columns=['index'])], axis=1)
per_distribution['resample_no'] = [s if s < ns else ns for (s, ns) in per_distribution[['superhost', 'non_superhost']].values]
per_distribution

,Age Group,superhost,non_superhost,resample_no
0,2,57,318,57
1,4,248,316,248
2,6,467,789,467
3,8,587,483,483
4,10,355,402,355
5,12,110,77,77
6,14,35,5,5


In [27]:
df_lists = []
for group_, resample_no  in per_distribution[['Age Group', 'resample_no']].values:
    ns_df = non_super_hosts_df[non_super_hosts_df.hosted_for_grouped == group_].sample(resample_no)
    s_df = super_hosts_df[super_hosts_df.hosted_for_grouped == group_].sample(resample_no)
    df_lists.append(ns_df.copy())
    df_lists.append(s_df.copy())

resampled_df = pd.concat(df_lists, ignore_index=True)

In [28]:
print("Resampled Distributions for the Hosted for feature. We can now compare the 2 groups using the p-values")
alt.Chart(resampled_df[['Hosted for', 'host_is_superhost']]).mark_bar().encode(x=alt.X('Hosted for', bin=True), y='count()', column='host_is_superhost:N')

Resampled Distributions for the Hosted for feature. We can now compare the 2 groups using the p-values


alt.Chart(...)

In [29]:
%%time
output = perform_permutation_test(resampled_df[resampled_df.host_is_superhost==True].income_per_month , resampled_df[resampled_df.host_is_superhost==False].income_per_month, x_axis_title='Monthly Income', chart_title='Monthly Income differences between Super-Hosts and non Super-Hosts')
output[1]

Our null hypothesis: The difference in mean values is due to random chance
Our test statistic is 932.2479
Group 1 has a proportion of 0.5000
The p-value for these 2 groups is 0.00000000
We reject the Null Hypothesis
Wall time: 15.3 s


alt.Chart(...)

## Does proximity to competition negatively impact your monthly AirBnb income?               
For this question, I define a listing that has many competitors as one that has more than 4 **_similar_** listings within 500M.

In [30]:
listings_merged_df['Many_Competitors'] = None
listings_merged_df.loc[listings_merged_df.dist_500m > 4, 'Many_Competitors'] = True
listings_merged_df.loc[listings_merged_df.Many_Competitors.isna(), 'Many_Competitors'] = False
listings_merged_df.Many_Competitors = listings_merged_df.Many_Competitors.astype(bool)

In [31]:
%%time
output = perform_permutation_test(listings_merged_df[listings_merged_df.Many_Competitors==False].income_per_month, listings_merged_df[listings_merged_df.Many_Competitors==True].income_per_month, x_axis_title='Monthly Income', chart_title='Monthly Income differences between listings with Many Competitors and those with few')
output[1]

Our null hypothesis: The difference in mean values is due to random chance
Our test statistic is 380.2736
Group 1 has a proportion of 0.7011
The p-value for these 2 groups is 0.00000000
We reject the Null Hypothesis
Wall time: 19.2 s


alt.Chart(...)

It does matter. Proximity to many competitors does have an impact on the listings revenues

## What are the key factors that influence the rate a host sets for their listing in Seattle?

In [32]:
modelling_df = listings_merged_df.copy()

In [33]:
modelling_df = modelling_df[['host_is_superhost', 'neighbourhood_group_cleansed',
       'property_type', 'room_type', 'accommodates', 'amen_TV_Entertainment',
       'amen_Sound_System', 'amen_WorkSpace', 'amen_Internet',
       'amen_Paid_Parking', 'amen_Free_Parking', 'amen_Clothing_Storage',
       'amen_Linens', 'amen_House_Keeping', 'amen_Bathroom_Essentials',
       'amen_Restuarant', 'amen_Breakfast', 'amen_Gym', 'amen_Pool_Sauna',
       'amen_Games', 'amen_Children_Babies', 'amen_Dryer_Washer',
       'amen_Backyard_Garden', 'amen_Beach_Water_Front', 'amen_Pets',
       'amen_Kitchen', 'amen_Kitchen_Appliances', 'amen_Cooking_Dishes',
       'amen_Host_Greeting', 'amen_Long_Term_Stay', 'amen_Indoor_Fireplace',
       'amen_Outdoor', 'amen_Air_Conditioning', 'amen_Heating',
       'amen_Safety_Security_First_Aid', 'amen_Staff', 'dist_500m', 'price']]

In [34]:
modelling_df.sample(5)

,host_is_superhost,neighbourhood_group_cleansed,property_type,room_type,accommodates,amen_TV_Entertainment,amen_Sound_System,amen_WorkSpace,amen_Internet,amen_Paid_Parking,amen_Free_Parking,amen_Clothing_Storage,amen_Linens,amen_House_Keeping,amen_Bathroom_Essentials,amen_Restuarant,amen_Breakfast,amen_Gym,amen_Pool_Sauna,amen_Games,amen_Children_Babies,amen_Dryer_Washer,amen_Backyard_Garden,amen_Beach_Water_Front,amen_Pets,amen_Kitchen,amen_Kitchen_Appliances,amen_Cooking_Dishes,amen_Host_Greeting,amen_Long_Term_Stay,amen_Indoor_Fireplace,amen_Outdoor,amen_Air_Conditioning,amen_Heating,amen_Safety_Security_First_Aid,amen_Staff,dist_500m,price
504,True,Central Area,Entire guesthouse,Entire home/apt,1,0,0,1,1,0,1,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,1,1,1,0,0,0,1,0,0,1,1250.0
2521,False,Central Area,Entire rental unit,Entire home/apt,3,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,3,0,0,0,1,1,0,0,1,0,0,0,1,4,0,2,88.0
2993,False,Lake City,Entire rental unit,Entire home/apt,2,2,0,1,1,0,1,1,1,1,2,0,0,0,0,0,2,1,1,0,0,1,2,1,0,1,0,0,0,1,4,0,1,119.0
3400,True,Other neighborhoods,Private room in residential home,Private room,2,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,3,0,0,0,1,0,1,1,1,0,0,0,1,5,0,15,320.0
1515,True,West Seattle,Entire guest suite,Entire home/apt,2,0,0,1,1,0,2,2,2,2,5,0,0,0,0,1,0,3,0,0,0,0,4,4,0,1,0,2,1,1,6,0,1,114.0


In [35]:
cat_cols_lst = list(modelling_df.select_dtypes(include=['object']).columns) + ['host_is_superhost']

mapped_dict = {}
for col in cat_cols_lst:
    mapped_dict[col] = LabelEncoder().fit(modelling_df[col])
    modelling_df[col] = mapped_dict[col].transform(modelling_df[col])

transformer = FunctionTransformer(np.log1p)
modelling_df.price = transformer.fit_transform(modelling_df.price)

Here I choose an LGBM model because:
1. It can handle Categorical variables without One-Hot-Encoding
1. No scaling or normalization is required on the feature set
1. It is non-linear
1. Traning time is very short

In [36]:
%%time
#Test your function with the above dataset
test_score, train_score, lm_model, X_train, X_test, y_train, y_test = clean_fit_lgbm_mod(modelling_df, 'price', cat_cols_lst, dummy_na=False, rand_state=42)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000666 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 248
[LightGBM] [Info] Number of data points in the train set: 2974, number of used features: 36
[LightGBM] [Info] Start training from score 4.900312
[1]	valid_0's l2: 0.349057
Training until validation scores don't improve for 10 rounds
[2]	valid_0's l2: 0.311175
[3]	valid_0's l2: 0.281004
[4]	valid_0's l2: 0.256117
[5]	valid_0's l2: 0.235463
[6]	valid_0's l2: 0.218048
[7]	valid_0's l2: 0.203353
[8]	valid_0's l2: 0.191702
[9]	valid_0's l2: 0.18209
[10]	valid_0's l2: 0.174647
[11]	valid_0's l2: 0.167717
[12]	valid_0's l2: 0.161854
[13]	valid_0's l2: 0.157376

E:\ProgramData\Miniconda3\envs\project_1_udacity\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
E:\ProgramData\Miniconda3\envs\project_1_udacity\lib\site-packages\lightgbm\basic.py:1706: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['host_is_superhost', 'neighbourhood_group_cleansed', 'property_type', 'room_type']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
E:\ProgramData\Miniconda3\envs\project_1_udacity\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
E:\ProgramData\Miniconda3\envs\project_1_udacity\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))



[14]	valid_0's l2: 0.153701
[15]	valid_0's l2: 0.151165
[16]	valid_0's l2: 0.148887
[17]	valid_0's l2: 0.147106
[18]	valid_0's l2: 0.145217
[19]	valid_0's l2: 0.143331
[20]	valid_0's l2: 0.141621
[21]	valid_0's l2: 0.140265
[22]	valid_0's l2: 0.13904
[23]	valid_0's l2: 0.138033
[24]	valid_0's l2: 0.137508
[25]	valid_0's l2: 0.13705
[26]	valid_0's l2: 0.136979
[27]	valid_0's l2: 0.136888
[28]	valid_0's l2: 0.136488
[29]	valid_0's l2: 0.136252
[30]	valid_0's l2: 0.136066
[31]	valid_0's l2: 0.135537
[32]	valid_0's l2: 0.135091
[33]	valid_0's l2: 0.135117
[34]	valid_0's l2: 0.134927
[35]	valid_0's l2: 0.135021
[36]	valid_0's l2: 0.13458
[37]	valid_0's l2: 0.134376
[38]	valid_0's l2: 0.134069
[39]	valid_0's l2: 0.13405
[40]	valid_0's l2: 0.134024
[41]	valid_0's l2: 0.133834
[42]	valid_0's l2: 0.134046
[43]	valid_0's l2: 0.134029
[44]	valid_0's l2: 0.133649
[45]	valid_0's l2: 0.133619
[46]	valid_0's l2: 0.133563
[47]	valid_0's l2: 0.133581
[48]	valid_0's l2: 0.133654
[49]	valid_0's l2: 0.13

In [37]:
#Print training and testing score
print("The rsquared on the training data was {}.  The rsquared on the test data was {}.".format(train_score, test_score))

The rsquared on the training data was 0.7913680919342095.  The rsquared on the test data was 0.6635108519111719.


In [38]:
importances_df = pd.DataFrame()
importances_df['Features'] = X_train.columns
importances_df['Importance_Values'] = lm_model.feature_importances_
importances_df = importances_df.sort_values(by=['Importance_Values'], ascending=False)
importances_df.Features = importances_df.Features.replace({'dist_500m':'competition', 'neighbourhood_group_cleansed': 'neighbourhood'})

alt.Chart(importances_df.iloc[:20]).mark_bar().encode(x=alt.X('Importance_Values:Q'), y=alt.Y('Features:N', sort='-x')).properties(title='Feature Importances')

alt.Chart(...)